In [21]:
import pandas as pd

# Replace 'file_path.csv' with the actual path to your CSV file
file_path = '/work/zonghaiyao_umass_edu/SALT/data/AVS/synthetic_data/hight_to_low_dic_5k.json'

# Read the CSV file into a DataFrame
train_df = pd.read_json(file_path)

# Now you can work with the DataFrame 'df'
train_df.head(2)  # Display the first few rows of the DataFrame

,article,chosen,rejected,hallucination_instructions
0,Article:Brief Hospital Course: Pt was given 3 ...,Discharge Instructions: Please call you doctor...,Discharge Instructions: \nPlease call your do...,Numbered List hallucination edits made:\n1. Ad...
1,Article:Brief Hospital Course: An 86-year-old ...,Discharge Instructions: You were admitted for ...,Discharge Instructions: \nAn 86-year-old woma...,Numbered List hallucination edits made:\n1. Ad...


In [22]:
def do_stats(x):
    return len(x.split())

df['len'] = df['chosen'].apply(do_stats)

In [23]:
df['len'].tolist()

[82,
 95,
 74,
 39,
 61,
 77,
 81,
 90,
 33,
 72,
 91,
 84,
 69,
 97,
 81,
 89,
 103,
 94,
 80,
 99,
 41,
 57,
 86,
 62,
 58,
 96,
 87,
 90,
 81,
 82,
 103,
 98,
 70,
 89,
 103,
 76,
 80,
 71,
 52,
 67,
 66,
 96,
 64,
 81,
 81,
 84,
 78,
 60,
 77,
 78,
 68,
 44,
 95,
 58,
 110,
 93,
 38,
 36,
 83,
 75,
 85,
 55,
 74,
 88,
 61,
 83,
 84,
 31,
 94,
 61,
 84,
 33,
 74,
 60,
 72,
 94,
 39,
 74,
 78,
 94,
 41,
 31,
 51,
 75,
 48,
 71,
 77,
 79,
 37,
 68,
 93,
 103,
 109,
 92,
 56,
 82,
 103,
 103,
 69,
 103,
 95,
 106,
 30,
 77,
 87,
 81,
 75,
 61,
 64,
 86,
 58,
 87,
 74,
 40,
 51,
 69,
 89,
 31,
 33,
 93,
 99,
 66,
 97,
 88,
 89,
 79,
 79,
 107,
 74,
 80,
 82,
 45,
 63,
 98,
 44,
 99,
 73,
 92,
 102,
 93,
 91,
 99,
 71,
 82,
 87,
 35,
 100,
 50,
 98,
 77,
 54,
 103,
 81,
 98,
 62,
 86,
 84,
 35,
 97,
 40,
 48,
 91,
 53,
 76,
 80,
 52,
 50,
 106,
 82,
 79,
 81,
 89,
 51,
 96,
 88,
 48,
 100,
 93,
 80,
 95,
 73,
 72,
 36,
 30,
 58,
 80,
 79,
 60,
 83,
 60,
 84,
 92,
 80,
 83,
 71,
 48,
 66,

In [24]:
# Remove duplicated rows based on the 'Summary' column
train_df = train_df.drop_duplicates(subset=['chosen'], keep='first')

In [25]:
file_path = '/work/zonghaiyao_umass_edu/SALT/data/AVS/valid.json'
eval_df = pd.read_json(file_path, lines=True)

file_path = '/work/zonghaiyao_umass_edu/SALT/data/AVS/test.json'
test_df = pd.read_json(file_path, lines=True)

In [26]:
train_df.to_csv('train.csv', index=False)
eval_df.to_csv('eval.csv', index=False)
test_df.to_csv('test.csv', index=False)

### DPO/SALT

In [30]:
import pandas as pd

train_df = pd.read_csv('train.csv')
eval_df = pd.read_csv('eval.csv')
test_df = pd.read_csv('test.csv')

In [31]:
train_df['chosen'] = train_df['article'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: " + train_df['chosen']
train_df['rejected'] = train_df['article'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: " + train_df['rejected']

eval_df['chosen'] = eval_df['text'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: " + eval_df['summary']
eval_df['rejected'] = eval_df['text'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: " + eval_df['summary']

test_df['chosen'] = test_df['text'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: "+ test_df['summary']
test_df['rejected'] = test_df['text'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: " + test_df['summary']

In [32]:
train_df = train_df[['chosen', 'rejected']]
eval_df = eval_df[['chosen', 'rejected']]
test_df = test_df[['chosen', 'rejected']]

In [33]:
sub_eval_df = eval_df.sample(n=128, random_state=42)
sub_eval_df

,chosen,rejected
1801,Brief Hospital Course: Mr. [**Known lastname 4...,Brief Hospital Course: Mr. [**Known lastname 4...
1190,Brief Hospital Course: Patient was admitted fr...,Brief Hospital Course: Patient was admitted fr...
1817,Brief Hospital Course: MEDICINE COURSE: Ms [**...,Brief Hospital Course: MEDICINE COURSE: Ms [**...
251,Brief Hospital Course: Mr. [**Known lastname 6...,Brief Hospital Course: Mr. [**Known lastname 6...
2505,Brief Hospital Course: Admitted same day surge...,Brief Hospital Course: Admitted same day surge...
...,...,...
1770,Brief Hospital Course: Mr. [**Known lastname *...,Brief Hospital Course: Mr. [**Known lastname *...
1200,Brief Hospital Course: Ms. [**Known lastname *...,Brief Hospital Course: Ms. [**Known lastname *...
2174,Brief Hospital Course: Mr. [**Known lastname 1...,Brief Hospital Course: Mr. [**Known lastname 1...
1991,Brief Hospital Course: Mr. [**Known lastname *...,Brief Hospital Course: Mr. [**Known lastname *...


In [34]:
from datasets import Dataset

# Convert pandas DataFrames to dictionaries
train_dict = train_df.to_dict(orient='list')
eval_dict = eval_df.to_dict(orient='list')
sub_eval_dict = sub_eval_df.to_dict(orient='list')
test_dict = test_df.to_dict(orient='list')

# Create datasets from dictionaries
train_dataset = Dataset.from_dict(train_dict)
eval_dataset = Dataset.from_dict(eval_dict)
sub_eval_dataset = Dataset.from_dict(sub_eval_dict)
test_dataset = Dataset.from_dict(test_dict)

/work/zonghaiyao_umass_edu/anaconda3/envs/salt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
# Replace 'dataset_directory' with the directory where you want to save your datasets
train_dataset.save_to_disk('train')
eval_dataset.save_to_disk('eval')
sub_eval_dataset.save_to_disk('sub_eval')
test_dataset.save_to_disk('test')

Saving the dataset (1/1 shards): 100%|██████████| 3082/3082 [00:00<00:00, 13537.71 examples/s]


### SFT

In [1]:
import pandas as pd

train_df = pd.read_csv('train.csv')
eval_df = pd.read_csv('eval.csv')
test_df = pd.read_csv('test.csv')

In [2]:
train_df = train_df.dropna()
eval_df = eval_df.dropna()
test_df = test_df.dropna()

In [4]:
train_df['chosen'] = train_df['article'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: " + train_df['chosen']
train_df['rejected'] = train_df['article'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: " + train_df['chosen']

eval_df['chosen'] = eval_df['text'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: " + eval_df['summary']
eval_df['rejected'] = eval_df['text'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: " + eval_df['summary']

test_df['chosen'] = test_df['text'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: "+ test_df['summary']
test_df['rejected'] = test_df['text'] + "\n\nGenerate the corresponding Discharge Instructions according to the input article: " + test_df['summary']

In [5]:
train_df = train_df[['chosen', 'rejected']]
eval_df = eval_df[['chosen', 'rejected']]
test_df = test_df[['chosen', 'rejected']]

In [6]:
sub_eval_df = eval_df.sample(n=128, random_state=42)
sub_eval_df

,chosen,rejected
1801,Brief Hospital Course: Mr. [**Known lastname 4...,Brief Hospital Course: Mr. [**Known lastname 4...
1190,Brief Hospital Course: Patient was admitted fr...,Brief Hospital Course: Patient was admitted fr...
1817,Brief Hospital Course: MEDICINE COURSE: Ms [**...,Brief Hospital Course: MEDICINE COURSE: Ms [**...
251,Brief Hospital Course: Mr. [**Known lastname 6...,Brief Hospital Course: Mr. [**Known lastname 6...
2505,Brief Hospital Course: Admitted same day surge...,Brief Hospital Course: Admitted same day surge...
...,...,...
1770,Brief Hospital Course: Mr. [**Known lastname *...,Brief Hospital Course: Mr. [**Known lastname *...
1200,Brief Hospital Course: Ms. [**Known lastname *...,Brief Hospital Course: Ms. [**Known lastname *...
2174,Brief Hospital Course: Mr. [**Known lastname 1...,Brief Hospital Course: Mr. [**Known lastname 1...
1991,Brief Hospital Course: Mr. [**Known lastname *...,Brief Hospital Course: Mr. [**Known lastname *...


In [7]:
from datasets import Dataset

# Convert pandas DataFrames to dictionaries
train_dict = train_df.to_dict(orient='list')
eval_dict = eval_df.to_dict(orient='list')
sub_eval_dict = sub_eval_df.to_dict(orient='list')
test_dict = test_df.to_dict(orient='list')

# Create datasets from dictionaries
train_dataset = Dataset.from_dict(train_dict)
eval_dataset = Dataset.from_dict(eval_dict)
sub_eval_dataset = Dataset.from_dict(sub_eval_dict)
test_dataset = Dataset.from_dict(test_dict)

/work/zonghaiyao_umass_edu/anaconda3/envs/salt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Replace 'dataset_directory' with the directory where you want to save your datasets
train_dataset.save_to_disk('ref_train')
eval_dataset.save_to_disk('ref_eval')
sub_eval_dataset.save_to_disk('ref_sub_eval')
test_dataset.save_to_disk('ref_test')

Saving the dataset (1/1 shards): 100%|██████████| 3082/3082 [00:00<00:00, 13361.74 examples/s]
